# Package Import

In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_nb_helper import ASPRules
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
from nxpd import nxpdParams
from nxpd import draw
nxpdParams['show'] = 'ipynb'
import pandas as pd
from ipywidgets import IntSlider
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# from graphviz import Source
import matplotlib.image as mpimg
from IPython.display import Image
from networkx.algorithms.dag import ancestors

In [2]:
%load_ext PWE_NB_Extension

# CAP Relationship

<img src="img/kcap_relationship.png"> 

**conflict(R1,R2)**: requirement R1 and R2 are conflict

Axiom: 
* has_conflict(R1,R2) ⇒ has_conflict(R2,R1) i.e., has_conflict/2 is symmetric
* has_conflict is not (in general) transitive! 
* A requirement can have multiple conflicts (“M-N relationship”)

**uses(M,R)**: Module M “needs” requirement R to run

Axiom:
* many-to-many relationship

**on(M,B)**: module M will be run on box (i.e., container) B

⇒ is it enough to install M only on one box? Or are there reasonable use cases to allow M to be installed on multiple boxes?

If we allow more complex applications (such as workflows that are built using multiple modules), then it might be desirable to install a module M on multiple boxes (as required / useful for the workflow)
But since we here consider M as the top-level application, we only need a single box to run M.
So we can assume here that the functional dependency M → B holds!

# M-R Instance

<img src="img/cap_instance.png"  width="600"> 

Take the above instance as an example, we can translate it into clingo script

In [3]:
%%clingo --donot-run --donot-display_input -lci instance
% instance

uses(m1,r1).
uses(m1,r4).
uses(m2,r1).
uses(m3,r2).
uses(m3,r4).
uses(m4,r2).
uses(m4,r3).
uses(m5,r2).

has_conflict(r1,r2).
has_conflict(r3,r4).

# CAP Generate

For each module, there could be multiple boxes to choose from while every module M should be associated with exactly one box B (see the figure below)

<img src="img/all_possible_cap.png" width="600"/> 

In [4]:
%%clingo --donot-run --donot-display_input -lci generate

mod(M) :- uses(M,_).
%% This works: for each M, choose exactly one B:
1 { on(M,B) : box(B) } 1  :- mod(M).

# ASP Check

In [5]:
%%clingo --donot-run --donot-display_input -lci check

inst(B,R) :- on(M,B), uses(M,R).
 :- has_conflict(R1,R2), inst(B,R1), inst(B,R2).

#show inst/2.

# Problem1: K_CAP

 Given a constant K, find all possible solutions to CAP with k containers

In [6]:
k=3

In [7]:
gen_ls=generate.splitlines()
k_str='box(1..'+str(k)+').'
gen_ls.insert(1, k_str) ## insert into cont_rules
generate_updated=ASPRules('\n'.join(gen_ls)) ## formalize the cont_rules

In [8]:
clingo_soln, meta_data = run_clingo(clingo_rules=instance+generate_updated+check)

In [9]:
pw_ref_dfs,rel_schemas,pw_objs=load_worlds(asp_output=clingo_soln,reasoner='clingo')

Number of Models: 12


In [10]:
pw_ref_dfs['inst_2']

,pw,x1,x2
0,1,1,r1
1,1,1,r4
2,1,2,r4
3,1,2,r2
4,1,3,r2
5,1,3,r3
6,2,1,r1
7,2,1,r4
8,2,2,r4
9,2,2,r2


# Problem2: min_K_CAP

In [16]:
k=0
success_sol_len=0
#find the minimum number of containers to satisfy all
while success_sol_len<=0:
    k=k+1  ## for other problems, the variables should be revised
    print("container number: "+str(k)) 
    gen_ls=generate.splitlines()
    box_str='box(1..'+str(k)+').'
    gen_ls.insert(1, box_str) ## insert into cont_rules
    generate_updated=ASPRules('\n'.join(gen_ls)) ## formalize the cont_rules
    # get solutions
    clingo_soln, meta_data = run_clingo(clingo_rules=instance+generate_updated+check)
    #analyze the solutions
    pw_ref_dfs,rel_schemas,pw_objs=load_worlds(asp_output=clingo_soln,reasoner='clingo')
    success_sol_len=len(pw_ref_dfs)

container number: 1
The problem is unsatisfiable
Number of Models: 0
container number: 2
The problem is unsatisfiable
Number of Models: 0
container number: 3
Number of Models: 12
